In [ ]:
!pip install gensim transformers torch nltk rouge-score tqdm pandas

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 63.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=295d52cbd672383d727e469cac390923d7391ee7005efdcd8e219f912f815a12
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import KeyedVectors
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
df = pd.read_csv('/content/news_summary_more.csv')

df

,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...
...,...,...
98396,CRPF jawan axed to death by Maoists in Chhatti...,A CRPF jawan was on Tuesday axed to death with...
98397,First song from Sonakshi Sinha's 'Noor' titled...,"'Uff Yeh', the first song from the Sonakshi Si..."
98398,'The Matrix' film to get a reboot: Reports,"According to reports, a new version of the 199..."
98399,Snoop Dogg aims gun at clown dressed as Trump ...,A new music video shows rapper Snoop Dogg aimi...


In [ ]:
def clean_text(text):
    text = str(text)
    text = text.replace('\n', ' ')
    text = text.replace('\r', '')
    text = text.strip()
    return text

df['text'] = df['text'].apply(clean_text)
df['headlines'] = df['headlines'].apply(clean_text)

In [ ]:
!wget -q https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
!gunzip cc.en.300.vec.gz

ft_model = KeyedVectors.load_word2vec_format('cc.en.300.vec', limit=500000)
print("FastText model loaded successfully!")

FastText model loaded successfully!


In [ ]:
def get_sentence_vector(sentence):
    words = word_tokenize(sentence.lower())
    word_vecs = [ft_model[w] for w in words if w in ft_model]
    if not word_vecs:
        return np.zeros(300)
    return np.mean(word_vecs, axis=0)

def extractive_summary(text, num_sentences=3):
    sentences = sent_tokenize(text)
    if len(sentences) <= num_sentences:
        return ' '.join(sentences)

    sent_vecs = [get_sentence_vector(s) for s in sentences]
    doc_vec = np.mean(sent_vecs, axis=0).reshape(1, -1)

    sims = [cosine_similarity(doc_vec, v.reshape(1, -1))[0][0] for v in sent_vecs]
    ranked = np.argsort(sims)[-num_sentences:]
    ranked = sorted(ranked)
    return ' '.join([sentences[i] for i in ranked])

In [ ]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print("Flan-T5 model loaded successfully!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Flan-T5 model loaded successfully!


In [ ]:
def hybrid_summary(text):
    extractive = extractive_summary(text)

    prompt = "Summarize this news article in one short headline:\n" + extractive
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)

    summary_ids = model.generate(
        **inputs,
        max_length=25,
        min_length=5,
        num_beams=4,
        length_penalty=1.5,
        early_stopping=True
    )

    headline = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return headline

In [ ]:
while True:
    print("\n Enter a news article (or type 'quit' to exit):")
    user_input = input().strip()

    if user_input.lower() == 'quit':
        print(" Exiting summarizer.")
        break

    if not user_input:
        print(" Please enter a valid article.")
        continue

    print("\n Generating summary...\n")
    summary = hybrid_summary(user_input)
    print(" Predicted Headline:\n")
    print(summary)
    print("-" * 100)


 Enter a news article (or type 'quit' to exit):
Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coins. Users get one CRED coin per rupee of bill paid, which can be used to avail rewards from brands like Ixigo, BookMyShow, UberEats, Cult.Fit and more.

 Generating summary...

 Predicted Headline:

A Delhi techie has been given a chance to win a free meal from Swiggy for one year
----------------------------------------------------------------------------------------------------

 Enter a news article (or type 'quit' to exit):
quit
 Exiting summarizer.


In [ ]:
df.loc[1,'text']

"Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coins. Users get one CRED coin per rupee of bill paid, which can be used to avail rewards from brands like Ixigo, BookMyShow, UberEats, Cult.Fit and more."

In [ ]:
df.loc[1,'headlines']

'Delhi techie wins free food from Swiggy for one year on CRED'